# Classification of Photo Taken with Cam

In [14]:
import cv2 as cv
import pickle as pk
import numpy as np
with open('let_KNN_clf.pickle','rb') as f:
    let_KNN_clf = pk.load(f)
with open('dig_KNN_clf.pickle','rb') as f:
    dig_KNN_clf = pk.load(f)
with open('dig_MLP_clf.pickle','rb') as f:
    dig_MLP_clf = pk.load(f)
with open('let_MLP_clf.pickle', 'rb') as f:
    let_MLP_clf = pk.load(f)
with open('bal_MLP_clf.pickle','rb') as f:
    bal_MLP_clf = pk.load(f)
with open('dig_MLP_clf_hog.pickle', 'rb') as f:
    dig_MLP_clf_hog = pk.load(f)


In [33]:
im = cv.imread('digits.jpg',cv.IMREAD_GRAYSCALE)
ret, im_th = cv.threshold(im, 90, 255, cv.THRESH_BINARY_INV)
cv.imshow('s', im_th)
cv.waitKey(0)
cv.destroyAllWindows()

In [86]:
def get_roi_all(image): 
    ''' find objects in image, return vector of segmented images and respective rectangles'''
    padding = 15
    # find contours
    image = cv.GaussianBlur(image,(5,5),1)
    ret, image = cv.threshold(image, 90, 255, cv.THRESH_BINARY_INV)
    ctrs, hier = cv.findContours(image.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    # find rectangles
    rects = [cv.boundingRect(ctr) for ctr in ctrs]
    #copy each object to image vector
    im_vec = [image[rect[1]-padding:rect[1]+rect[3]+padding, rect[0]-padding:rect[0]+rect[2]+padding] for rect in rects]
    return im_vec, rects

def draw_named_rect(image, rectangle, text):
    '''draw rectangle and name it'''
    rect = rectangle
    img = image
    cv.rectangle(img, (rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]), 0, 1)
    cv.putText(img, text,(rect[0],rect[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.9,0, 1)
    return img
      
def adjust_img(image, width, height):
    ''' apply lowpass filter to image and resize to (height, width)'''
    image = cv.resize(image, dsize = (height,width),interpolation=cv.INTER_AREA)
    image_vec = image.reshape(1,-1)
    return image_vec

def pred_character(classifier, im_vec, target):
    '''classifies object and returns ascii code'''
    #dictionary to look up class and character
    balanced_dict = dict(zip([i for i in range(0,47)],
                        [str(i) for i in range(0,10)]+['A','B','C','D','E','F','G',
                                                      'H','I','J','K','L','M','N','O',
                                                      'P','Q','R','S','T','U','V','W',
                                                      'X','Y','Z']+['a','b','d','e','f',
                                                                    'g','h','n','q','r','t']))
    offset = 0
    pred  = classifier.predict(np.array(im_vec, 'float64'))
   # import pdb
   # pdb.set_trace()
    # set character depending on target_type
    if target== 'digits' or target == 'all':
        pred_char = balanced_dict[pred[0]]
    elif target == 'letters':
        #classes from 1 to 26 (upper + lower case in one class)
        offset = 64 # A --> 65
    else:
        raise ValueError('target must be "digits", "all" or "letters"')
    return pred, pred_char

def start(clf, target_type):
    '''predict digit,and display classification
    '''
    im = cv.imread('letters_digits.jpg',cv.IMREAD_GRAYSCALE)
    # find the character in image
    images_roi, rects = get_roi_all(im)

    for idx, rect in enumerate(rects):
        # fit the image to data set
        im_vec = adjust_img(images_roi[idx], 28 ,28)
        #predict and return ascii code
        pred, pred_char = pred_character(clf, im_vec, target_type)
        #draw classification
        #im = draw_named_rect(im, rect, chr(pred_ascii))
        im = draw_named_rect(im, rect, pred_char)
    cv.imshow('s', im)
    cv.waitKey(0)
    cv.destroyAllWindows()


In [87]:
start(bal_MLP_clf, 'all')

In [77]:
balanced_dict = dict(zip([i for i in range(0,47)],
                        [str(i) for i in range(0,10)]+['A','B','C','D','E','F','G',
                                                      'H','I','J','K','L','M','N','O',
                                                      'P','Q','R','S','T','U','V','W',
                                                      'X','Y','Z']+['a','b','d','e','f',
                                                                    'g','h','n','q','r','t']))
balanced_dict

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: 'A',
 11: 'B',
 12: 'C',
 13: 'D',
 14: 'E',
 15: 'F',
 16: 'G',
 17: 'H',
 18: 'I',
 19: 'J',
 20: 'K',
 21: 'L',
 22: 'M',
 23: 'N',
 24: 'O',
 25: 'P',
 26: 'Q',
 27: 'R',
 28: 'S',
 29: 'T',
 30: 'U',
 31: 'V',
 32: 'W',
 33: 'X',
 34: 'Y',
 35: 'Z',
 36: 'a',
 37: 'b',
 38: 'd',
 39: 'e',
 40: 'f',
 41: 'g',
 42: 'h',
 43: 'n',
 44: 'q',
 45: 'r',
 46: 't'}